# 👩‍🚀**<span style="color:#4527A0;"> Spaceship Titanic with</span> <span style="color:#F44336;"> LGBMClassifier </span>🌠**

# <span style="color:#4527A0;"> Project </span> 🔥
- 강의명 : 2022년 K-디지털 직업훈련(Training) 사업 - AI데이터플랫폼을 활용한 빅데이터 분석전문가 과정
- 교과목명 : 빅데이터 분석 및 시각화, AI개발 기초, 인공지능 프로그래밍
- 프로젝트 주제 : Spaceship Titanic 데이터를 활용한 탑승유무 분류모형 개발
- 프로젝트 마감일 : 2022년 4월 12일 화요일
- 강사명 : Evan
- 수강생명 : 이가영

# <span style="color:#4527A0;"> Table of content </span> 📋

- [Load data](#Load-Data)
- [EDA](#EDA)
- [Preprocessing](#Preprocessing)
- [Model](#Model)

 # Load Data
- 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

- 데이터 불러오기

In [2]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

In [3]:
train.shape, test.shape, submission.shape

((8693, 14), (4277, 13), (4277, 2))

## Train Data

In [4]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


## Test Data

In [8]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [10]:
test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [11]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


결측치값들도 많고 RoomService와 같은 유흥비 요약을 보면 특이점이 있어 시각화로 확인할 필요성이 있다.

## EDA

**<span style="color:#4527A0;"> 데이터 특성 분석 </span>**
- PassengerId : 승객 아이디 ( 그룹 넘버 - 그룹 내 승객 번호) 
- HomePlanet : 승객이 출발한 행성
- CryoSleep : 승객이 항해 기간 동안 애니메이션을 일시 중단하도록 선택했는지 여부를 나타냅니다.
- Cain : 객실
- Destination : 승객이 도착한 행성
- Age : 나이
- VIP : VIP 해당 여부
- RoomService ~ VRDeck : 유흥비 
- Name : 이름
- Transported : target

- 행성 출발지(**HomePlanet**)와 행성 도착치(**Destination**)의 비율을 알아본다.

In [12]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2,shared_yaxes=True)
    
fig.add_trace(go.Scatter( x=train['HomePlanet'].value_counts().index, y=train['HomePlanet'].value_counts().values,
                         mode='markers',name = 'HomePlanet',
        marker=dict(
            sizemode = 'diameter',
            sizeref = 25,
            size = train['HomePlanet'].value_counts().values)),1,1)
    
fig.add_trace(go.Scatter( x=train['Destination'].value_counts().index, y=train['Destination'].value_counts().values,
                         mode='markers', name = 'Destination',
        marker=dict(
            sizemode = 'diameter',
            sizeref = 25,
            size = train['Destination'].value_counts().values)),1,2)


fig.update_layout(    title='<b>Planets</b>', 
        title_x=0.5,
        titlefont=dict(size =20, color='black', family='Space Mono'),
        plot_bgcolor='rgba(0,0,0,0)' )
fig.show()

출발지는 **Earth**가 제일 많고 도착지는 **TRAPPIST-1e**가 압도적으로 많다.

- **CryoSleep**에 따른 **Transported** 여부

In [13]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

colors = ['rgb(230,218,135)', 'rgb(228,139,133)','rgb(227,178,175)']

fig.add_trace(go.Pie(labels = train.CryoSleep, name = 'CryoSleep'),1,1)
fig.add_trace(go.Pie(labels = train.VIP, name = 'VIP'),1,2)

fig.update_traces(hole=.4, textinfo="label+percent",marker_colors = colors)

fig.update_layout(
    title_text="CryoSleep And VIP",
    annotations=[dict(text='CryoSleep', x=0.17, y=0.5, font_size=20, showarrow=False),
                 dict(text='VIP', x=0.8, y=0.5, font_size=20, showarrow=False)])

fig.show()

- 연령대 분포 (**Age**)

In [14]:
fig = go.Figure()

fig.add_trace(go.Histogram( x = train.Age, marker_color = 'rgb(228,139,133)'))

fig.show()

주로 20대가 많이 사용한 걸 확인 할 수 있다.

- 연령대별 Transported 여부

In [15]:
t = train[['Transported','Age']].copy()
          
x0 = t[t['Transported'] == True]
x1 = t[t['Transported'] == False]

In [16]:
fig = go.Figure()
fig.add_trace(go.Histogram( x = x0.Age, name ='Transported YES',
                           marker_color = '#657B96'))
fig.add_trace(go.Histogram( x = x1.Age, name ='Transported NO',
                           marker_color = '#E6B250'))

fig.update_layout( bargap = 0.3)

fig.show()

0세 ~ 17세 : Transported Yes가 우세  
18세 ~ 39세 : Transported No가 우세하거나 비등하다.  
40세 ~ : 비등하다.

- 유흥비 ( **RoomService, FoodCourt, ShoppingMall, Spa, VRDeck** )

In [17]:
fig = go.Figure()
fig.add_trace(go.Box( y = train['RoomService'],name = 'RoomService'))
fig.add_trace(go.Box( y = train['FoodCourt'], name = 'FoodCourt'))
fig.add_trace(go.Box( y = train['ShoppingMall'], name = 'ShoppingMall'))
fig.add_trace(go.Box( y = train['Spa'], name = 'Spa'))
fig.add_trace(go.Box( y = train['VRDeck'], name = 'VRDeck'))
fig.show()

극 소수 인원이 RoomService, FoodCourt, ShoppingMall, Spa,VRDeck을 사용했다.  

---

# Preprocessing

### PassengerId
 -> 단체 승객끼리 그룹으로 나눠서 단체 승객 인원 수를 구한다.

- PassengerId의 앞자리 4개를 이용해 단체 승객 그룹 Id 생성 -> 컬럼 Group으로 데이터 값 채우기

In [18]:
train['Group'] = train['PassengerId'].apply(lambda x : str(x[:4]))
test['Group'] = test['PassengerId'].apply(lambda x : str(x[:4]))
train[['PassengerId','Group']].head()

,PassengerId,Group
0,0001_01,0001
1,0002_01,0002
2,0003_01,0003
3,0003_02,0003
4,0004_01,0004


In [19]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
Group            object
dtype: object

PassengerId의 앞의 4자리가 잘 분리 되었고 Object 형태로 저장이 되었다.

- Group을 이용해 단체 승객 인원수를 구한다.

In [20]:
train['GroupSize'] = train.groupby('Group')['Group'].transform('count')
test['GroupSize'] = test.groupby('Group')['Group'].transform('count')
train[['Group','GroupSize']].sort_values(['Group']).head()

,Group,GroupSize
0,0001,1
1,0002,1
2,0003,2
3,0003,2
4,0004,1


In [21]:
train['GroupSize'].describe()

count    8693.000000
mean        2.035546
std         1.596347
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max         8.000000
Name: GroupSize, dtype: float64

단체 승객 수는 1인 부터 최대 8인 단체까지 존재한다.

### Name

처음에는 PassegerId의 그룹 번호가 가족단위라고 생각할 수 있다.  
하지만 우리가 친구끼리 여행이나 비지니스차 이동하는 경우도 많다는 걸 생각할 수 있다.  
그래서 PassengerId로는 단체 승객으로 구분하고 Name을 통해서 가족을 구분한다.  

- **Name**을 통해서 **LastName**에 이름의 성 저장

In [22]:
train['LastName'] = train['Name'].str.split(' ').str[-1]
test['LastName'] = test['Name'].str.split(' ').str[-1]

In [23]:
train['LastName']

0         Ofracculy
1             Vines
2            Susent
3            Susent
4       Santantines
           ...     
8688      Noxnuther
8689      Mondalley
8690         Connon
8691      Hontichre
8692      Hontichre
Name: LastName, Length: 8693, dtype: object

### Cabin 나누기

- cd : Cabin Deck
- cn : Cabin Number
- cs : Cabin Side

In [24]:
train['cd'] = train['Cabin'].str.split('/').str[0]
train['cn'] = train['Cabin'].str.split('/').str[1]
train['cs'] = train['Cabin'].str.split('/').str[2]

test['cd'] = test['Cabin'].str.split('/').str[0]
test['cn'] = test['Cabin'].str.split('/').str[1]
test['cs'] = test['Cabin'].str.split('/').str[2]

### Encoding

사용할 라이브러리 불러오기

In [25]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
 14  Group         8693 non-null   object 
 15  GroupSize     8693 non-null   int64  
 16  LastName      8493 non-null   object 
 17  cd            8494 non-null   object 
 18  cn            8494 non-null 

컬럼 중에 범주형 컬럼들만 따로 label_cols 변수로 저장한다.

In [27]:
label_cols= ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP","cd","cs","Name","LastName","PassengerId","Group"]

- LabelEncoder
    - 범주형 컬럼들을 수치형 데이터들로 바꿔주는 작업

In [28]:
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] = LabelEncoder().fit_transform(test[col])
    return train, test
    
train, test = label_encoder(train,test,label_cols)

### 결측치 처리하기

- SimpleImputer
    - 수치형 컬럼의 결측치 평균값으로 채워주기
    - 범주형 컬럼의 결측치 최빈값으로 채워주기

In [29]:
train.isna().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age             179
VIP               0
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name              0
Transported       0
Group             0
GroupSize         0
LastName          0
cd                0
cn              199
cs                0
dtype: int64

결측치 처리하기 전 많은 결측치들이 있는 것을 확인 할 수 있다.

In [30]:
imputer_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP","cd","cs","LastName"]
STRATEGY = 'most_frequent'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

print("train_data:\n", train[imputer_cols].isnull().sum())
print(20*"-")
print("test:\n", test[imputer_cols].isnull().sum())

train_data:
 HomePlanet     0
CryoSleep      0
Destination    0
VIP            0
cd             0
cs             0
LastName       0
dtype: int64
--------------------
test:
 HomePlanet     0
CryoSleep      0
Destination    0
VIP            0
cd             0
cs             0
LastName       0
dtype: int64


In [31]:
train['cs'].unique()

array([0, 1, 2])

In [32]:
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService","GroupSize","cn"]
STRATEGY = 'median'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

print("train_data:\n", train[imputer_cols].isnull().sum())
print(20*"-")
print("test:\n", test[imputer_cols].isnull().sum())

train_data:
 Age             0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
RoomService     0
GroupSize       0
cn              0
dtype: int64
--------------------
test:
 Age             0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
RoomService     0
GroupSize       0
cn              0
dtype: int64


📍 결측치 제거 완료

ML에 사용하지 않을 컬럼들을 분리시킨다.

In [33]:
train = train.drop(['Cabin','PassengerId','Name','Group'],axis = 1)
test = test.drop(['Cabin','PassengerId','Name','Group'],axis = 1)

---

# Model

In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   int64  
 1   CryoSleep     8693 non-null   int64  
 2   Destination   8693 non-null   int64  
 3   Age           8693 non-null   float64
 4   VIP           8693 non-null   int64  
 5   RoomService   8693 non-null   float64
 6   FoodCourt     8693 non-null   float64
 7   ShoppingMall  8693 non-null   float64
 8   Spa           8693 non-null   float64
 9   VRDeck        8693 non-null   float64
 10  Transported   8693 non-null   bool   
 11  GroupSize     8693 non-null   float64
 12  LastName      8693 non-null   int64  
 13  cd            8693 non-null   int64  
 14  cn            8693 non-null   float64
 15  cs            8693 non-null   int64  
dtypes: bool(1), float64(8), int64(7)
memory usage: 1.0 MB


📍 **TARGET** 데이터인 **Transported** 빼고 ML을 위한 사전 준비 작업이 완료되었다.

- ML에 사용할 특성만 col에 저장하고 TARGET인 Transported 특성만 TARGET 변수에 저장한다

In [35]:
col = train.drop(['Transported'], axis = 1).columns.tolist()
TARGET = 'Transported'

- ML 하기 전 훈련 데이터와 테스트 데이터 세트 분리

In [36]:
X = train[col]
Y = train[TARGET]
X_train, y_train, X_val, y_val = train_test_split(X,Y,test_size = 0.33)

- StratifiedKFold
    - 5번 교차 검증을 진행한다.
    - 진행 결과를 idx_fold에 저장한다.

In [37]:
idx_fold = []

skf = StratifiedKFold(5, random_state=12, shuffle=True)
for idx_train, idx_valid in skf.split(X = X, y = Y):
    idx_fold.append((idx_train, idx_valid))

- 하이퍼파라미터 저장

In [38]:
lgb_params = {
    'application' : 'binary',
    'n_estimators' : 975,
    'min_child_samples' : 5,
    'max_depth' : 8,
    'learning_rate' : 0.01,
    'n_jobs' : -1,
    'importance_type' : 'gain'
}

max_depth : 트리의 최대 깊이 지정  
application : 모델의 어플리케이션 지정 ( regression 회귀 , binary 이진, multiclass 다중 )  
learning_rate : 추정값 추정 중에서 발생하는 변화의 크기 컨트롤(각 트리에 영향을 주는 변수)  
n_estimators : 적절한 트리 수 지정  
min_child_samples : 리프 트리에서 최소 필요한 노드 개수 지정 (과적합 해결 파라미터)  
n_jobs : 훈련에 사용할 병렬 스레드 수 (학습 시간 줄여준다.)  
importance_type : 기능 중요도 유형 지정 ( gain = 사용한 feature 분할의 총 이득 포함, split = model에서 사용된 feature의 수 포함 )
  
참고 문서 : [공식문서](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html)

- 검증 점수와 예측을 저장할 변수 설정

In [39]:
lgb_scores = []
lgb_auc = []
predictions = 0

- idx_fold값에 따라 5번 모델을 돌린다.
- AUC의 값이 높을 수록 좋은 모델이다.

In [40]:
from sklearn.metrics import roc_auc_score

In [41]:
for fold, (idx_train, idx_valid) in enumerate(idx_fold):
    print(10*"=", f"Fold={fold+1}", 10*"=")
    
    X_train, X_valid = train.iloc[idx_train][col],train.iloc[idx_valid][col]
    Y_train, Y_valid = train.iloc[idx_train][TARGET],train.iloc[idx_valid][TARGET]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, Y_train, verbose = 0)
    
    # 정확도 측정해서 저장
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(Y_valid, preds_valid)
    lgb_scores.append(acc)
    roc = roc_auc_score(Y_valid, model.predict_proba(X_valid)[:, 1])
    lgb_auc.append(roc)
    
    preds = model.predict(test[col]) 
    predictions += preds/5

    
    print(f"Fold={fold+1}, Accuracy score : {acc :2f}%")
    print(f"Fold={fold+1}, AUC : {roc :2f}%\n")
    

print("Mean Accuracy : ", np.mean(lgb_scores))
print("Mean AUC : ", np.mean(lgb_auc))

========== Fold=1 ==========
Fold=1, Accuracy score : 0.815411%
Fold=1, AUC : 0.908927%

========== Fold=2 ==========
Fold=2, Accuracy score : 0.796435%
Fold=2, AUC : 0.896463%

========== Fold=3 ==========
Fold=3, Accuracy score : 0.806786%
Fold=3, AUC : 0.894971%

========== Fold=4 ==========
Fold=4, Accuracy score : 0.798044%
Fold=4, AUC : 0.900287%

========== Fold=5 ==========
Fold=5, Accuracy score : 0.813579%
Fold=5, AUC : 0.902101%

Mean Accuracy :  0.8060507904030662
Mean AUC :  0.9005498661732348


AUC는 0.9인 모델이며 평균 정확도는 0.806이다.

---

# Submission

In [42]:
submission[TARGET] = predictions.astype("bool")
submission.to_csv("submission.csv",index=False)
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


# 참고

[Spaceship Titanic - EDA + 27 different models](https://www.kaggle.com/code/odins0n/spaceship-titanic-eda-27-different-models#Data-Pre-Processing--)  

# 결과
4/4 : accuracy : 0.791   |  등수 :   341 / 전체  

4/5 : accuracy : 0.80102    |    등수  :  323  / 전체  

4/6 : accuracy : 0.80243    |    등수  :  292  / 1187  

4/7 : accuracy : 0.8043    |    등수  :  210  / 1226 
